# Unit Testing: How we make sure our code doesn't break

Unit testing is something that software engineers love to a borderline unhealthy level. If you ever want a SE to talk at length, just mention unit testing. However, they aren't wrong to be smitten with the concept. So, what is unit testing?

Unit testing is the idea that we want to make sure our functions always do what we think they do. In order to do that, we come up with a series of tests where we know what the function SHOULD do, and then make sure it does that. Let's start with a hand-coded, silly example.

In [ ]:
def add_two_to_int(input_int):
    return input_int + 2

add_two_to_int(4) # I know this should result in 6

Great, it did the thing! However, if I put this code out to my team and someone typos or screws things up, will we know later? Right now, we'd have to come back in and manually check things out. So let's write a little function that will automate that testing.

In [ ]:
def test_add_two_to_int():
    assert add_two_to_int(4) == 6, "Test 1 failed!"
    assert add_two_to_int(4.7) == 6, "Test 2 failed!"
    return "Passed!"
    
test_add_two_to_int()

We failed a test! Why is that? We can see that our version of the function doesn't enforce that the input must be an integer. So we need to clean that up. Let's re-write the function.

In [ ]:
def add_two_to_int(input_int):
    return int(input_int) + 2

add_two_to_int(4.7) # I know this should result in 6

In [ ]:
test_add_two_to_int()

That's unit testing in a nutshell. We're going to write a bunch of tests that make sure that our code works the way we think it does. And then every time an update is made to the code, we'll re-run the tests to make sure all the functions still do what they're supposed to.

There are a few concepts to consider when writing tests:

**What are the edge cases?**

In the above example, `4.7` was an edge case. It isn't something that was an obvious way the function would break. When writing tests, you need to think about all of the ways the code might ever be accessed. Do you need to handle types differently? Are there kwargs that might get weird? What happens if the user puts in something really strange?

**How should your code break?**

One of the things we didn't do above was check to see if the code breaks gracefully. As an example, if the user puts in a `string` where we expect an `int` we want to make sure the code appropriately raises a `TypeError`. That's hard to do in the "handmade" version of testing we did above. So let's introduce a better library called `unittest`.

In [ ]:
import unittest

`unittest` assumes we're going to build a class of tests that work on something. Let's take a look at the canonical example from the `unittest` documentation. We're going to test a bunch of string methods. 

First, we design a class that inherits from the `TestCase` class that has all of the major functionality. Each test we build is named `test_WHATEVER_THE_TEST_DOES`. 

In [ ]:
class TestStringMethods(unittest.TestCase):

    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)

Now we actually tell `unittest` to get to work and see if anything breaks. 

> The kwargs in `unittest.main()` are because we're running this in a Jupyter Notebook. Normally this would be run from the command line. 

In [ ]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

Hooray! Everything is OK. Let's break it really quickly. Go to the line in `test_isupper` and change `assertFalse` to `assertTrue` then re-run those cells. You should see it break and tell you exactly where it broke.  

## Exercise: Building and testing an anagram finder

I'm writing below a unit testing class. It will help you decode what you need a function to do. Your goal is to fill in the function below to make it pass all of those tests. After this, we'll move on to bigger fish.

In [ ]:
from collections import defaultdict

def check_if_anagrams(input1, input2):
    if type(input1) not in [int, str, float]:
        raise TypeError("Must be int, str, or float inputs")
    if type(input2) not in [int, str, float]:
        raise TypeError("Must be int, str, or float inputs")
    in1_dict = defaultdict(int)
    in2_dict = defaultdict(int)
    for character in str(input1).lower():
        in1_dict[character] += 1
    for character in str(input2).lower():
        in2_dict[character] += 1
    return in1_dict == in2_dict

In [ ]:
class TestAnagrams(unittest.TestCase):
    
    def test_output_type(self):
        self.assertIs(type(check_if_anagrams("tar","rat")), bool)

    def test_strings(self):
        self.assertTrue(check_if_anagrams("elvis","lives"))
        self.assertFalse(check_if_anagrams("prince","lives"))

    def test_ints(self):
        self.assertTrue(check_if_anagrams(1234,4321))
        self.assertFalse(check_if_anagrams(1234,5321))

    def test_floats(self):
        self.assertTrue(check_if_anagrams(12.34,4.213))
        self.assertFalse(check_if_anagrams(1.234,53.21))
        
    def test_spaces(self):
        self.assertTrue(check_if_anagrams("clint eastwood","old westaction"))
        self.assertFalse(check_if_anagrams("clint eastwood","old west action"))
        
    def test_list_and_tuple_errors(self):
        with self.assertRaises(TypeError):
            check_if_anagrams([1,2,3,4],[2,3,4,1])
        with self.assertRaises(TypeError):
            check_if_anagrams("steve",(2,3,4,1))
        with self.assertRaises(TypeError):
            check_if_anagrams((2,3,4,1), 1101)
            
    def test_capitals(self):
        self.assertTrue(check_if_anagrams("TEST","test"))
        self.assertTrue(check_if_anagrams("ClInT EaStWooD","old westaction"))
        

In [ ]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)